Trap Depth Calculations

In [1]:
import numpy as np
import scipy.constants as sc

In [2]:
def to_freq_ang(wavelength: float) -> float:
    """Converts a wavelength to its angular frequency using the formula 
       omega = 2*pi*c / lambda

    Args:
        wavelength (float): Wavelength [nm]

    Returns:
        float: Angular frequency in s^-1
    """

    # Converts a wavelength in nanometers of light to radians
    return (2*np.pi*sc.speed_of_light*1e9)/(wavelength)

In [3]:
def alpha(wavelength: float) -> complex:
    """Calculates the polarizability alpha using equation (8) from the Grimm review paper

    Args:
        wavelength (float): wavelength [nm]

    Returns:
        complex: Complex polarizability alpha
    """
    # Units = (A^2s^4)/(kg rad)
    # Use Equation (8)

    # https://jet.physics.ncsu.edu/techdocs/pdf/PropertiesOfLi.pdf

    omega_0       = to_freq_ang(wavelength = 671)
    omega         = to_freq_ang(wavelength = wavelength)
    gamma_omega_0 = 36.898e6  
    # Line width of the transition (We are only interested in the damping there)

    result  = 6 * np.pi * sc.epsilon_0 * (sc.speed_of_light**3)
    result *= gamma_omega_0 / (omega_0**2)
    result /= complex(omega_0**2 - omega**2, - (omega**3/omega_0**2)*gamma_omega_0)

    return result


In [4]:
def potential(intensity: float, wavelength: float) -> float:
    """Calculates the dipole potential using equation (2) of the Grimms Review Paper

    Args:
        intensity (float): Intensity   [W/m^2]
        wavelength (float): Wavelength [nm]

    Returns:
        float: Potential 
    """
    
    # Equation (2) of Grimm
    _alpha = alpha(wavelength = wavelength)
    print("alpha =", _alpha)

    U_dip  = - np.real(_alpha) * intensity
    U_dip /= 2 * sc.epsilon_0 * sc.speed_of_light

    return U_dip

In [5]:
def trap_temperature(trap_depth: float) -> float:
    """Converts a trap depth (i.e. min dipole potential) to a temperature using T = U/kB

    Args:
        trap_depth (float): Trap Depth [J]

    Returns:
        float: Temperature of the trap
    """
    
    return -trap_depth/sc.Boltzmann

In [6]:
from typing import Tuple

def gaussian_beam_width(z: float, w_0: float , z_0: float, Msq: float, wavelength: float) -> float:
    """ Returns the gaussian beam width based on a gaussian beam propagation
    
    Note that this function is normalized if:
	- Everything is in SI-Units, or
	- w, w_0: [um], z, z_0: [mm], lmbda: [nm] (preferred)


    Args:
        z (float): Position in propagation direction [m, mm]
        w_0 (float): Beam waist [m, um]
        z_0 (float): Position of beam waist [m, mm]
        Msq (float): M-squared beam quality factor
        wavelength (float): Wavelength of light [m, nm]

    Returns:
        float: _description_
    """

    return w_0 * np.sqrt(
		1 + ((z - z_0)**2)*((
			(Msq * wavelength)/
			(np.pi * (w_0**2))
		)**2)
	)

def max_intensity(power: float, width_x: float, width_y: float) -> float:
    """Generate the max-intensity of the gaussian intensity distribution given power, w_x and w_y

    Function is normalized if:
    - Everything in SI-units, or
    - P [W], w_x [um], w_y [um] (preferred)

    Args:
        power (float): Power [W]
        width_x (float): width [m, um]
        width_y (float): width [m, um]

    Returns:
        float: Max intensity I_0 in [W/m^2, W/um^2]
    """
    return (2 * power) / (np.pi * (width_x*width_y))

def intensity(
        x: float, y: float, z: float,
        power: float, 
        wavelength: float,
        w_0: Tuple[float, float],
        z_0: Tuple[float, float],
        Msq: Tuple[float, float]
    ) -> float:

    """Returns the intensity of a guassian beam at a point in 3D space. Assumes a simple astigmatic beam

    Note that this function is normalized if:
	- Everything is in SI-Units, or
	- w, w_0: [um], z, z_0: [mm], lmbda: [nm] (preferred)

    Args:
        x (float): x position (one of the main axes)                        [m, um]
        y (float): y position (one of the main axes)                        [m, um]
        z (float): z position (propagation direction)                       [m, mm]
        power (float): Power of the beam                                    [W]
        wavelength (float): Wavelength of the light                         [m, nm]
        w_0 (Tuple[float, float]): Tuple of the beam waist (x, y axes)      [m, um]
        z_0 (Tuple[float, float]): Tuple of the rayleigh length (x, y axes) [m, mm]
        Msq (Tuple[float, float]): Tuple of the beam quality factor M^2 (x, y axes)

    Returns:
        float: The intensity at that point [W/m^2, W/um^2]
    """ 

    # the x and y are the main axes for astigmatism

    x_params = { "w_0": w_0[0], "z_0": z_0[0], "Msq": Msq[0], "wavelength": wavelength }
    y_params = { "w_0": w_0[1], "z_0": z_0[1], "Msq": Msq[1], "wavelength": wavelength }

    w_x = gaussian_beam_width(z = z, **x_params)
    w_y = gaussian_beam_width(z = z, **y_params)

    I0 = max_intensity(power = power, width_x = w_x, width_y = w_y)

    intensity = I0 * np.exp(-2*((x/w_x)**2 + (y/w_y)**2))

    return intensity

# Check units

In [7]:
x = gaussian_beam_width(z = 3,    w_0 = 5, z_0 = 6, Msq = 1, wavelength = 1070)
y = gaussian_beam_width(z = 3e-3, w_0 = 5e-6, z_0 = 6e-3, Msq = 1, wavelength = 1070e-9)

print(x, y * 1e6) # convert to um

204.4161058594956 204.41610585949562


In [8]:
x = max_intensity(power = 100, width_x = 60, width_y = 60)
y = max_intensity(power = 100, width_x = 40e-6, width_y = 40e-6)
print(x, y * 1e-12) # convert to W/um^2

0.01768388256576615 0.03978873577297383


# Calculations

In [9]:
U = potential(intensity = x*1e12, wavelength = 1070) # SI UNIT OUTPUT
temp = trap_temperature(U)

print(U, "J", temp*1e3, "mK")

alpha = (4.403504782680971e-39+2.352531047837446e-47j)
-1.466819470370436e-26 J 1.0624130176246358 mK


# Combining multiple Gaussians

In [10]:
def transform_matrix_angle(axis: np.ndarray, degrees: float) -> np.ndarray:
    """Produce a transformation matrix to transform from one coordinate system to another (Beam 1 vs Beam 2).
    Use the inverse to do a back-transformation

    Reference: https://en.wikipedia.org/wiki/Transformation_matrix#Rotation_2

    To be applied to a (1x3) row vector. r' = rT. Should be extensible to (nx3)

    Args:
        degrees (float): Angle to rotate along the x-z plane in degrees

    Returns:
        np.ndarray: Transformation matrix
    """

    # unit vector representing the rotation axis
    unit_axis = axis.flatten()
    unit_axis = unit_axis / np.linalg.norm(unit_axis)
    l,m,n = unit_axis[0], unit_axis[1], unit_axis[2]

    angle = np.deg2rad(degrees)

    cos  = np.cos(angle, dtype = np.float64)
    cosp = (1 - cos)
    sin  = np.sin(angle, dtype = np.float64)

    # without .T, it is for r' = Tr with r as a column vector
    ret = np.array([
        [l*l*cosp+cos  , m*l*cosp-n*sin, n*l*cosp+m*sin],
        [l*m*cosp+n*sin, m*m*cosp+cos  , n*m*cosp-l*sin],
        [l*n*cosp*m*sin, m*n*cosp+l*sin, n*n*cosp+cos  ]
    ]).T

    return ret

In [11]:
from scipy.spatial.transform import Rotation

def transform_quaternion_angle(axis: np.ndarray, degrees: float) -> Rotation: 
    halfangle = np.deg2rad(degrees)/2

    unit_axis = axis.flatten()
    unit_axis = unit_axis / np.linalg.norm(unit_axis)

    ax = np.sin(halfangle) * unit_axis
    q  = np.concatenate(([np.cos(halfangle)], ax))

    return Rotation.from_quat(q)

In [12]:
import quaternion

def get_quaternion_angle(axis: np.ndarray, degrees: float) -> np.ndarray: 
    halfangle = np.deg2rad(degrees)/2

    unit_axis = axis.flatten()
    unit_axis = unit_axis / np.linalg.norm(unit_axis)

    ax = np.sin(halfangle) * unit_axis
    q_arr = np.concatenate(([np.cos(halfangle)], ax)).tolist()
    q  = np.quaternion(*q_arr)

    return q

In [13]:
xps = [np.array([3, 0, 0])]

x = np.array([0, 0, 3])
T = transform_matrix_angle(axis = np.array([0,1,0]), degrees = 90)
xp = np.matmul(x, T)

xps.append(xp)

print(x, xp)

[0 0 3] [3.0000000e+00 0.0000000e+00 1.8369702e-16]


In [14]:
R = transform_quaternion_angle(axis = np.array([0,1,0]), degrees = 90)
xp = R.apply(x)

xps.append(xp)

print(x, xp)

[0 0 3] [3. 0. 0.]


In [15]:
q  = get_quaternion_angle(axis = np.array([0,1,0]), degrees = 90)

x  = np.quaternion(0, 0, 0, 3)
xp = q*x*q.conjugate()

xps.append(np.array([xp.x, xp.y, xp.z]))

print(x, xp)

quaternion(0, 0, 0, 3) quaternion(0, 3, 0, 0)


In [16]:
xps

[array([3, 0, 0]),
 array([3.0000000e+00, 0.0000000e+00, 1.8369702e-16]),
 array([3., 0., 0.]),
 array([3., 0., 0.])]